In [5]:
import pandas as pd
import numpy as np
import requests
from urllib.parse import urlencode

In [18]:
# используем api 
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' 
public_key = 'https://disk.yandex.ru/d/PBW7aUHGuodFDA'
public_key1 = 'https://disk.yandex.ru/d/m0Z6QYNT46f9tQ'
public_key2 = 'https://disk.yandex.ru/d/lsmdbYB0iM7p3w'
public_key3 = 'https://disk.yandex.ru/d/Yse4Y6RJqg_WaA'
 
# получаем url 
final_url = base_url + urlencode(dict(public_key=public_key)) 
response = requests.get(final_url) 
download_url = response.json()['href'] 
final_url1 = base_url + urlencode(dict(public_key=public_key1))
response1 = requests.get(final_url1)
download_url1 = response1.json()['href']
final_url2 = base_url + urlencode(dict(public_key=public_key2)) 
response2 = requests.get(final_url2) 
download_url2 = response2.json()['href'] 
final_url3 = base_url + urlencode(dict(public_key=public_key3)) 
response3 = requests.get(final_url3) 
download_url3 = response3.json()['href']
 
# загружаем файлы 
download_response = requests.get(download_url) 
assessments = pd.read_csv(download_url) 
download_response1 = requests.get(download_url1) 
courses = pd.read_csv(download_url1) 
download_response2 = requests.get(download_url2) 
studentAssessment = pd.read_csv(download_url2)
download_response3 = requests.get(download_url3) 
studentRegistration = pd.read_csv(download_url3)



# Задача 1:
    Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене)
    
    1. Из studentAssessment выделить тех, кто успешно сдал ( набрали больше 40 баллов).
    2. Приджойнить к studentAssessment типы ассесментов из файла assessments.
    3. Отфильтровать тех, которые относсятся к типу Exam.
    4. Посчитать количество уникальных успешно сданных экзаменов для каждого студента.
    5. Посчитать количество студентов, у которых успешно сдан только 1 экз.

In [52]:
task_1 = studentAssessment.query('score > 39') \
    .merge(assessments.query('assessment_type == "Exam"')[['id_assessment']].drop_duplicates(), on='id_assessment') \
    .groupby('id_student', as_index=False) \
    .agg({'id_assessment': 'nunique'}) \
    .rename(columns= {'id_assessment': 'successes_count'}) \
    .query('successes_count == 1') \
    .id_student.nunique()

In [58]:
print(task_1)

3802


# Задача 2
    Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене)
    
    1. Из assessments выделить тех, которые являются экзаменами.
    2. Приджойнить к studentAssessment типы ассесментов из файла assessments.
    3. Отфильтровать только те ассесменты, которые относятся к типу Exam.
    4. Посчитать количество уникальных успешных попыток для каждого экзамена.
    5. Посчитать количество неуспешных попыток.
    6. Вввести новое поле - завершаемость.
    7. Вывести ид асессментов - самого сложного и самого простого.
    
    

In [82]:
task_2 = studentAssessment \
    .merge(assessments.query('assessment_type == "Exam"')[['id_assessment']].drop_duplicates(), on='id_assessment') \
    .assign(success = lambda x: x.score > 39).astype(int) \
    .groupby('id_assessment', as_index=False) \
    .agg({'success': (sum, len)}) \
    .assign(success_rate = lambda row: row.success['sum']/row.success.len) \
    .sort_values('success_rate') \
    .reset_index(drop=True) \
    .id_assessment

In [84]:
print(f'самый сложный: {task_2[0]}\nсамый простой: {task_2[len(task_2)-1]}')

самый сложный: 25340
самый простой: 25361


# Задача 3
    Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью
    
    

In [87]:
task_3 = studentAssessment \
    .merge(assessments.query('assessment_type == "Exam"')[['id_assessment', 'code_module']].drop_duplicates(), on='id_assessment') \
    .query('is_banked == 0') \
    .groupby('code_module', as_index=False) \
    .agg({'date_submitted': 'mean'})

In [88]:
task_3

,code_module,date_submitted
0,CCC,239.408877
1,DDD,237.901445


# Задача 4
    Выяви самые популярные предметы (ТОП-3) по количеству регистраций на них. А также предметы с самым большим оттоком (ТОП-3).
    
    1. Для каждого семестра в таблице studentRegistration рассчитать уникальное количество зарегестрировавшихся
    2. Для каждого семестра в таблице studentRegistration рассчитать уникальное количество отписавшихся

In [97]:
task_4_top3 = studentRegistration.groupby('code_module', as_index=False) \
    .id_student.nunique() \
    .sort_values('id_student', ascending=False) \
    .rename(columns={'id_student': 'registration_count'}) \
    .head(3)
    
task_4_churn_top3 = studentRegistration[~studentRegistration.date_unregistration.isna()] \
    .groupby('code_module', as_index=False) \
    .id_student.nunique() \
    .sort_values('id_student', ascending=False) \
    .rename(columns={'id_student':'unregistrations_count'}) \
    .head(3)

In [98]:
task_4_top3

,code_module,registration_count
1,BBB,7692
5,FFF,7397
3,DDD,5848


In [99]:
task_4_churn_top3

,code_module,unregistrations_count
1,BBB,2314
5,FFF,2249
3,DDD,2065


# Задание 5
    Напиши функцию на python, позволяющую строить когортный (семестровый) анализ. В период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов.
    
    1. Из таблицы studentAssessment выделить те экзамены, которые не были зачтены ранее.
    2. Рассчитать на основе поля score факт успешности экзамена.
    3. Для успешных экзаменов выделить средние сроки их сдачи.
    4. Для каждого семестра рассчитать среднюю завершаемость и средние сроки сдачи

In [117]:
task_5 = studentAssessment.query('is_banked == 0') \
    .merge(assessments.query('assessment_type == "Exam"')[['id_assessment', 'code_presentation']].drop_duplicates(), on='id_assessment') \
    .assign(success = lambda x: (x.score > 39).astype(int),
            success_date_submitted = lambda x: x.success * x.date_submitted) \
    .groupby('code_presentation', as_index=False) \
    .agg({'success': 'mean',
          'success_date_submitted': [lambda x: np.average(x, weights=(x > 0))]})
task_5.columns = task_5.columns.droplevel(level=1)

In [118]:
task_5

,code_presentation,success,success_date_submitted
0,2013B,0.837209,230.164683
1,2013J,0.907025,239.509112
2,2014B,0.904013,232.997389
3,2014J,0.878659,243.680279


# Задание 6
    Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров.
    
    1. Из studentAssessment выделить только те экзамены, которые не были перезачтены
    2. Приджойнить к StudentAssessment (типы ассессментов из файла assessments)
    3. Отфильтровать только те ассесменты, которые относятся к типу Exam
    4. Для каждого студента рассчитать его средние : завершаемость, кол-во баллов, время сдачи одного экзамена
    5. Получить интервалы по 25, 50 и 75 процентилям для каждой метрики
    6. Заметим, что по завершаемости имеется только 2 кластера: среднее больше 1 или меньше 1 - скорректируем интервалы
    7. Разместим студентов по их кластерам

In [123]:
task_6_preproc = studentAssessment.query('is_banked == 0') \
    .merge(assessments.query('assessment_type == "Exam"')[['id_assessment', 'code_presentation']].drop_duplicates(), on='id_assessment') \
    .assign(success = lambda x: (x.score > 39).astype(int),
            success_date_submitted = lambda x: x.success * x.date_submitted) \
    .groupby('id_student', as_index=False) \
    .agg({'success': 'mean',
          'success_date_submitted': 'mean',
          'score': 'mean'})

intervals = {
    'R': list(task_6_preproc.success_date_submitted.describe())[3:],
    'F': list(task_6_preproc.success.describe())[3:5],
    'M': list(task_6_preproc.score.describe())[3:]
}

task_6 = task_6_preproc.assign(
    recency = lambda x: np.digitize(x.success, intervals.get('R')),
    frequance = lambda x: np.digitize(x.success_date_submitted, intervals.get('F')),
    monetary = lambda x: np.digitize(x.score, intervals.get('M')))[['id_student', 'recency', 'frequance', 'monetary']]

In [124]:
task_6

,id_student,recency,frequance,monetary
0,23698,1,2,3
1,24213,1,2,2
2,27116,1,2,4
3,28046,1,2,1
4,28787,1,2,1
...,...,...,...,...
4628,2694886,1,2,3
4629,2694933,1,2,3
4630,2695608,1,2,3
4631,2697181,1,2,3
